In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import nengo
from nengo.utils.matplotlib import rasterplot
from nengo.utils.ensemble import tuning_curves
%load_ext nengo.ipynb

<IPython.core.display.Javascript object>

In [ ]:
w_res = []
out_res = []

for theta_tau in (0.5, 1.0, 1.5):

    with nengo.Network() as model:
        sin = nengo.Node(lambda t: np.sin(t * 4))

        pre = nengo.Ensemble(4, 1, encoders=[[1], [1], [-1], [-1]], intercepts=[-0.1, 0, 0, -0.1], max_rates=[250, 300, 300, 250])
        post = nengo.Ensemble(2, 1, encoders=[[1], [-1]], intercepts=[-0.05, -0.05], max_rates=[200, 200])

        nengo.Connection(sin, pre)
        conn = nengo.Connection(
            pre, post, solver=nengo.solvers.LstsqL2(weights=True))

        conn.learning_rule_type = nengo.BCM(learning_rate=5e-10, theta_tau=theta_tau)

        p_weights = nengo.Probe(conn, 'weights', synapse=0.01, sample_every=0.01)
        p_out = nengo.Probe(post, synapse=0.01)
        
    with nengo.Simulator(model) as sim:
        sim.run(10)